In [1]:
import numpy as np
from scipy.special import logsumexp
import scipy.sparse as sp
from utils import *

In [62]:
class MC_PDDM(object):
    """
    Markov chain approximation of two Brownian motions with drift,
    whose drift are mu1 and mu2 respectively
    At each step, the process has a small probability of jumping from one to the other.
    """

    def __init__(self, mu1, mu2, sigma, lbda, a, z, t_nd, dt, Nx) -> None:
        self.mu1 = mu1
        self.mu2 = mu2
        self.sigma = sigma
        self.lbda = lbda
        self.a = a  # boundaries start at 'a' and '-a'
        self.z = -a + z * (2 * a)  # starting point
        self.t_nd = t_nd  # non-decision time

        # space and time discretization
        self.dt = dt
        self.Nx = Nx  # num of space steps
        dx = (2 * a) / Nx
        self.dx = dx
        # starting point and initial distribution
        self.idx_z = self.pos2idx(self.z)  # index of starting point
        self.init_dist = np.zeros(2 * self.Nx + 3)
        self.init_dist[self.idx_z] = 0.5
        self.init_dist[self.idx_z + self.Nx + 1] = 0.5

        # transitional probabilities
        m11 = mu1 * self.dt
        m21 = (mu1 * self.dt) ** 2 + self.sigma ** 2 * self.dt
        self.p11 = (m21 / self.dx ** 2 + m11 / self.dx) / 2
        self.p21 = (m21 / self.dx ** 2 - m11 / self.dx) / 2
        assert self.p11 + self.p21 < 1, "p+=%.5f, p0=%.5f, p-=%.5f" % (
            self.p11,
            1 - self.p11 - self.p21,
            self.p21,
        )
        m12 = mu2 * self.dt
        m22 = (mu2 * self.dt) ** 2 + self.sigma ** 2 * self.dt
        self.p12 = (m22 / self.dx ** 2 + m12 / self.dx) / 2
        self.p22 = (m22 / self.dx ** 2 - m12 / self.dx) / 2
        assert self.p12 + self.p22 < 1, "p+=%.5f, p0=%.5f, p-=%.5f" % (
            self.p12,
            1 - self.p12 - self.p22,
            self.p22,
        )
        self.Construct_AdjMat()

    def pos2idx(self, x):
        """
        find the nearest spatial grid point of 'x'
        return the index
        """
        return int(round((x + self.a) / self.dx))

    def Construct_AdjMat(self):
        """
        transition probability matrix
        0, 1, ..., Nx are transient states of BM1
        Nx+1, Nx+2, ..., 2Nx+1 are transient states of BM2
        2Nx+2 is the absorbing state
        arguments:
        t - time
        """
        MCTransProb = sp.dok_matrix((2 * self.Nx + 3, 2 * self.Nx + 3))
        nz_dict = {
            (0, 2 * self.Nx + 2): 1,
            (self.Nx, 2 * self.Nx + 2): 1,
            (self.Nx + 1, 2 * self.Nx + 2): 1,
            (2 * self.Nx + 1, 2 * self.Nx + 2): 1,
            (2 * self.Nx + 2, 2 * self.Nx + 2): 1,
        }
        for i in range(1, self.Nx):
            nz_dict[(i, i - 1)] = self.p21
            nz_dict[(i, i)] = 1 - self.p11 - self.p21
            nz_dict[(i, i + 1)] = self.p11
        for i in range(self.Nx + 2, 2 * self.Nx + 1):
            nz_dict[(i, i - 1)] = self.p22
            nz_dict[(i, i)] = 1 - self.p12 - self.p22
            nz_dict[(i, i + 1)] = self.p12
        dict.update(MCTransProb, nz_dict)
        with np.printoptions(precision=3, suppress=True):
            print(MCTransProb.toarray())

        PoissonTransProb = sp.dok_matrix((2 * self.Nx + 3, 2 * self.Nx + 3))
        nz_dict = {(2 * self.Nx + 2, 2 * self.Nx + 2): 1}
        for i in range(0, 2 * self.Nx + 2):
            nz_dict[(i, i)] = 1 - np.exp(-self.lbda * self.dt) * np.sinh(
                self.lbda * self.dt
            )
            nz_dict[(i, (i + self.Nx + 1) % (2 * self.Nx + 2))] = np.exp(
                -self.lbda * self.dt
            ) * np.sinh(self.lbda * self.dt)
        dict.update(PoissonTransProb, nz_dict)
        with np.printoptions(precision=3, suppress=True):
            print(PoissonTransProb.toarray())
        
        self.AdjMat = sp.csr_matrix(np.dot(PoissonTransProb, MCTransProb))

    def ExitDist(self, T):
        """
        compute the full distribution of X[T]
        where T is the first passage time
        by MATRIX MULTIPLICATION
        """
        T = T - self.t_nd
        dist_Xt = self.init_dist
        idx_T = int(round(T / self.dt))
        for t_step in range(idx_T):
            dist_Xt = dist_Xt @ self.AdjMat
        return dist_Xt
    
    def TotalExitProb(self, T, s):
        """
        return the exit probability of the original process
        """
        return self.ExitProb_dp(T, s) + self.ExitProb_dp(T, s, reflected=True) 

    def ExitProb_dp(self, T, s, reflected=False):
        """
        compute the probability of P(X[T]=s)
        where t is the first passage time
        by DYNAMIC PROGRAMMING based on SPARSE ADJACENCY MATRIX
        s: value in [-a, a]
        """
        T = T - self.t_nd
        idx_T = int(round(T / self.dt))
        idx_s = self.pos2idx(s)
        if reflected:
            idx_s += self.Nx + 1
        table = np.zeros((2 * self.Nx + 3, idx_T))
        table[:, [idx_T - 1]] = self.AdjMat[:, [idx_s]].toarray()
        for t_step in range(idx_T - 2, -1, -1):
            table[:, [t_step]] = self.AdjMat @ table[:, [t_step + 1]]
        return table[:, 0] @ self.init_dist

    def ExitProb_logdp(self, T, s, reflected=False):
        """
        compute the probability of P(X[T]=s) with a EXP scaling
        where t is the first passage time
        by DYNAMIC PROGRAMMING based on ADJACENCY MATRIX
        s: value in [-a, a]
        """
        T = T - self.t_nd
        idx_T = int(round(T / self.dt))
        idx_s = self.pos2idx(s)
        if reflected:
            idx_s += self.Nx + 1
        scaled_table = np.zeros((2 * self.Nx + 3, idx_T))
        r = 0
        scaled_table[:, [idx_T - 1]] = self.AdjMat[:, [idx_s]].toarray() / np.exp(r)
        for t_step in range(idx_T - 2, -1, -1):
            b = np.sum(self.AdjMat @ scaled_table[:, [t_step + 1]])
            r = r + np.log(b)
            scaled_table[:, [t_step]] = self.AdjMat @ scaled_table[:, [t_step + 1]] / b
        return scaled_table[:, 0] * np.exp(r) @ self.init_dist


In [63]:
mc = MC_PDDM(mu1=1, mu2=-1, sigma=1, lbda=2, a=1, z=0.5, t_nd=0.5, dt=0.1, Nx=4)

[[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.  ]
 [0.12 0.56 0.32 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.12 0.56 0.32 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.12 0.56 0.32 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.  ]
 [0.   0.   0.   0.   0.   0.32 0.56 0.12 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.32 0.56 0.12 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.32 0.56 0.12 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.  ]]
[[0.835 0.    0.    0.    0.    0.165 0.    0.    0.    0.    0.   ]
 [0.    0.835 0.    0.    0.    0.    0.165 0.    0.    0.    0.   ]
 [0.    0.    0.835 0.    0.    0.    0.    0.165 0.    0.    0.   ]
 [0.    0.    0.    0.835 0.    0.    0.    0.    0.165 0.    0.   ]
 [0.    0.    0.    0.    0.835 0.    0.    0.    0.    0.165 0.   ]
 [0.165 0.    0.

In [64]:
mc.ExitDist(T=1)

array([0.00936177, 0.05972627, 0.11398894, 0.10894222, 0.03781383,
       0.03781383, 0.10894222, 0.11398894, 0.05972627, 0.00936177,
       0.34033395])

In [65]:
mc.TotalExitProb(T=1, s=-mc.a)

0.047175599830472696

In [61]:
x_grid = np.linspace(-mc.a, mc.a, mc.Nx+1)
dist2 = [mc.ExitProb_dp(1, x_grid[i]) for i in range(len(x_grid))]
dist4 = [mc.ExitProb_logdp(1, x_grid[i]) for i in range(len(x_grid))]
dist2, dist4

([0.009361769890923376,
  0.05972626655646718,
  0.11398893890875335,
  0.10894222123552176,
  0.037813829939549314],
 [0.009361769890923377,
  0.05972626655646719,
  0.11398893890875333,
  0.10894222123552175,
  0.03781382993954932])

In [13]:
with np.printoptions(precision=4, suppress=True):
    print(mc.AdjMat.toarray())

[[0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.  ]
 [0.12 0.56 0.32 0.   0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.12 0.56 0.32 0.   0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.12 0.56 0.32 0.   0.   0.   0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.  ]
 [0.   0.   0.   0.   0.   0.32 0.56 0.12 0.   0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.32 0.56 0.12 0.   0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.32 0.56 0.12 0.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.  ]
 [0.   0.   0.   0.   0.   0.   0.   0.   0.   0.   1.  ]]


In [30]:
import pickle

filename = 'data_ih2.pkl'
with open('../Data/' + filename, "rb") as fp:
    pkl = pickle.load(fp)
    


In [33]:
rt_data, drift_data = pkl[0], pkl[1]
start_idx, end_idx = 0, 1000
rt_data = rt_data[start_idx: end_idx]
drift_data = drift_data[start_idx: end_idx]
print('%s %d: %d' %(filename, start_idx, end_idx))
print('data shape:' + str(rt_data.shape))
print()

data_ih2.pkl 0: 1000
data shape:(1000, 2)



In [32]:
intervals

[(array([ 0.        ,  0.13973244,  0.71524817,  2.17583838,  3.07584638,
          3.34243981,  3.50831275,  3.84286496,  4.26514135,  7.27093114,
          7.42748703,  8.24203893, 10.2899712 , 10.70707786, 10.94654558,
         12.19593392, 12.40818606, 14.22951829, 14.62984687, 15.83172827,
         16.54109991, 16.89772812, 17.53994083, 17.59538039, 17.75260018,
         17.77363222, 19.69268521, 22.46364288, 23.4401124 , 23.63879781,
         23.8201114 , 25.99456636, 26.25108733, 28.09887119, 28.27101064,
         28.30889766, 30.        ]),
  0),
 (array([ 0.        ,  1.75304862,  2.48006218,  4.78116896,  6.60234402,
          7.70458264,  7.70489433,  8.79251781,  9.83792897, 10.58240605,
         10.70181686, 12.10748885, 12.1400792 , 12.92898862, 14.02492732,
         14.10666884, 14.28854761, 15.48385918, 15.72085557, 19.21166054,
         22.43081538, 22.43896338, 23.69142834, 24.61030841, 26.23495876,
         27.90319234, 29.8674317 , 30.        ]),
  1),
 (array([ 0. 